# Group Project / Assignment 3: Retrieval-Augmented Generation Question Answering
**Assignment due 6 April 11:59pm 2025**

Welcome to the third assignment for 50.055 Machine Learning Operations. 
The third and fourth assignment together form the course group project. You will be working in your project groups to build a chatbot which can answer questions about SUTD to prospective students.


**This assignment is a group assignment.**

- Read the instructions in this notebook carefully
- Add your solution code and answers in the appropriate places. The questions are marked as **QUESTION:**, the places where you need to add your code and text answers are marked as **ADD YOUR SOLUTION HERE**
- The completed notebook, including your added code and generated output will be your submission for the assignment.
- The notebook should execute without errors from start to finish when you select "Restart Kernel and Run All Cells..". Please test this before submission.
- Use the SUTD Education Cluster to solve and test the assignment. If you work on another environment, minimally test your work on the SUTD Education Cluster.

**Rubric for assessment** 

Your submission will be graded using the following criteria. 
1. Code executes: your code should execute without errors. The SUTD Education cluster should be used to ensure the same execution environment.
2. Correctness: the code should produce the correct result or the text answer should state the factual correct answer.
3. Style: your code should be written in a way that is clean and efficient. Your text answers should be relevant, concise and easy to understand.
4. Partial marks will be awarded for partially correct solutions.
5. Creativity and innovation: in this assignment you have more freedom to design your solution, compared to the first assignments. You can show of your creativity and innovative mindset. 
6. There is a maximum of 225 points for this assignment.

**ChatGPT policy** 

If you use AI tools, such as ChatGPT, to solve the assignment questions, you need to be transparent about its use and mark AI-generated content as such. In particular, you should include the following in addition to your final answer:
- A copy or screenshot of the prompt you used
- The name of the AI model
- The AI generated output
- An explanation why the answer is correct or what you had to change to arrive at the correct answer

**Assignment Notes:** Please make sure to save the notebook as you go along. Submission instructions are located at the bottom of the notebook.



### Retrieval-Augmented Generation (RAG) 

In this assignment, you will be building a Retrieval-Augmented Generation (RAG) question answering system which can answer questions about SUTD.

We'll be leveraging `langchain` and `llama 3.2`.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [Llama 3.2](https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2/)


The SUTD website used to allow chatting with current students. Unfortunately, this feature does not exist anymore. Let's build a chatbot to fill this gap!


### Conduct user research

What are the questions that prospective and current students have about SUTD? In week 2, you already conducted some user research to understand your users.

### Value Proposition Canvas


### QUESTION: 

Paste the value proposition canvas which you have created in week 2 into this notebook below. 


**--- ADD YOUR SOLUTION HERE (10 points) ---**

- (replace canvas image below)

------------------------------


![VPC.png](VPC.png)

# Install dependencies
Use pip to install all required dependencies of this assignment in the cell below. Make sure to test this on the SUTD cluster as different environments have different software pre-installed.  

In [2]:
# QUESTION: Install and import all required packages
# The rest of your code should execute without any import or dependency errors.

# **--- ADD YOUR SOLUTION HERE (10 points) ---**
!pip install unstructured langchain pdfminer.six pi_heif unstructured-inference pdf2image tesseract pymupdf sentence-transformers faiss-cpu accelerate rank-bm25 nltk langchain-community
import os
import requests
from urllib.parse import urljoin, urlparse, quote
from collections import deque
import hashlib
import tqdm
import time
from bs4 import BeautifulSoup
from langchain.document_loaders import UnstructuredHTMLLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.schema import Document, BaseRetriever
from huggingface_hub import login
import torch
import fitz
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import re
from typing import Any, List
# ----------------

# Download documents
The RAG application should be able to answer questions based on ingested documents. For the SUTD chatbot, download PDF and HTML files from the SUTD website. The documents should contain information about the admission process, available courses and the university in general.


In [ ]:
# QUESTION: Download documents from the SUTD website
# You should download at least 10 documents but more documents can increase the knowledge base of your chatbot.

# **--- ADD YOUR SOLUTION HERE (20 points) ---**
# Creating a safe filename to download
def create_filename(url, extension=".html"):
    safe_url = quote(url, safe=":/")
    safe_url = safe_url.replace("https://", "").replace("http://", "")
    safe_url = safe_url.replace("/", "_").replace(":", "_")
    return f"{safe_url}{extension}"

# Checking the validity of the URL
def is_valid(url, base_netloc):
    parsed = urlparse(url)
    return bool(parsed.netloc) and (parsed.netloc.endswith(base_netloc) or base_netloc in parsed.netloc)

# Download PDF
def download_pdf(pdf_url, output_folder):
    try:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            filename = create_filename(pdf_url, extension=".pdf")
            file_path = os.path.join(output_folder, filename)
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Saved PDF: {filename}")
    except Exception as e:
        print(f"Error saving {pdf_url}: {e}")

# Find the next page
def find_next_page(soup):
    next_page = None
    for link in soup.find_all('a', href=True):
        if 'next' in link.get_text().lower():
            next_page = link['href']
            break
    return next_page

# Main crawling function
def crawl(url, base_netloc, visited, output_folder, depth=0, max_depth=2):
    if url in visited or depth > max_depth:
        return
    visited.add(url)
    
    try:
        response = requests.get(url)
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return
    
    content_type = response.headers.get('Content-Type', '')
    
    if 'application/pdf' in content_type:
        download_pdf(url, output_folder)
        return

    if 'text/html' not in content_type:
        return
    
    filename = create_filename(url, extension=".html")
    file_path = os.path.join(output_folder, filename)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"Saved webpage: {filename}")
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    next_page = find_next_page(soup)
    if next_page:
        next_url = urljoin(url, next_page)
        if is_valid(next_url, base_netloc):
            crawl(next_url, base_netloc, visited, output_folder, depth + 1, max_depth)

    for link in soup.find_all('a', href=True):
        next_url = urljoin(url, link['href'])
        if is_valid(next_url, base_netloc):
            crawl(next_url, base_netloc, visited, output_folder, depth + 1, max_depth)

output_folder = 'downloaded_docs'
os.makedirs(output_folder, exist_ok=True)

# URLs to crawl
urls_to_crawl = [
    "https://www.sutd.edu.sg/admissions",
    "https://www.sutd.edu.sg/admissions/undergraduate/",
    "https://www.sutd.edu.sg/admissions/graduate/masters/",
    "https://www.sutd.edu.sg/admissions/graduate/phd/",
    "https://www.sutd.edu.sg/admissions/academy",
    "https://www.42singapore.sg/admissions/",
    "https://www.sutd.edu.sg/innovation/",
    "https://www.sutd.edu.sg/education/accreditation",
    "https://www.sutd.edu.sg/research/research-programmes-fellowships",
    "https://www.sutd.edu.sg/innovation/educational-technology",
    "https://www.sutd.edu.sg/innovation/fabrication-lab",
    "https://www.sutd.edu.sg/campus-life",
    "https://www.sutd.edu.sg/about/happenings",
    "https://www.sutd.edu.sg/about/at-a-glance",
    "https://www.sutd.edu.sg/about/diversity-inclusion/building-gender-diversity/",
    "https://www.sutd.edu.sg/education/undergraduate/academic-calendar/overview/ay2024-onwards/",
    "https://www.sutd.edu.sg/education/undergraduate/academic-calendar/term-dates/2025-2/#tabs",
    "https://www.sutd.edu.sg/campus-life/student-life",
    "https://www.sutd.edu.sg/campus-life/academic-facilities/",
    "https://www.sutd.edu.sg/campus-life/housing/",
    "https://www.sutd.edu.sg/campus-life/sports-and-recreation-centre/",
    "https://www.sutd.edu.sg/campus-life/fnb-and-services/dining",
    "https://www.sutd.edu.sg/education/undergraduate/courses",
    "https://www.sutd.edu.sg/asd/",
    "https://www.sutd.edu.sg/dai/",
    "https://www.sutd.edu.sg/epd/",
    "https://www.sutd.edu.sg/esd/",
    "https://www.sutd.edu.sg/istd/",
    "https://www.sutd.edu.sg/smt/",
    "https://www.sutd.edu.sg/hass/"
    "https://www.sutd.edu.sg/education/undergraduate/majors",
    "https://www.sutd.edu.sg/education/undergraduate/minors",
]

visited = set()
for start_url in urls_to_crawl:
    base_netloc = urlparse(start_url).netloc
    crawl(start_url, base_netloc, visited, output_folder, depth=0, max_depth=3)

Saved webpage: www.sutd.edu.sg_admissions.html
Saved webpage: www.sutd.edu.sg_.html
Saved webpage: www.sutd.edu.sg_education_undergraduate_academic-calendar_.html
Saved webpage: www.sutd.edu.sg_about_people_faculty.html
Saved webpage: www.sutd.edu.sg_about_partnering-with-sutd.html
Saved webpage: www.sutd.edu.sg_resources.html
Saved webpage: www.sutd.edu.sg_education.html
Saved webpage: www.sutd.edu.sg_education_undergraduate.html
Saved webpage: www.sutd.edu.sg_admissions_graduate_masters_.html
Saved webpage: www.sutd.edu.sg_admissions_academy_.html
Saved webpage: www.sutd.edu.sg_education_accreditation.html
Saved webpage: www.sutd.edu.sg_education_cyber-physical-learning-alliance_.html
Saved webpage: www.sutd.edu.sg_education_undergraduate_courses.html
Saved webpage: www.sutd.edu.sg_asd.html
Saved webpage: www.sutd.edu.sg_dai.html
Saved webpage: www.sutd.edu.sg_epd.html
Saved webpage: www.sutd.edu.sg_esd.html
Saved webpage: www.sutd.edu.sg_hass.html
Saved webpage: www.sutd.edu.sg_istd

In [ ]:
# Scraping undergraduate course websites
def crawl_course_page(course_url, output_folder):
    try:
        response = requests.get(course_url)
    except Exception as e:
        print(f"Failed to fetch {course_url}: {e}")
        return
    
    content_type = response.headers.get('Content-Type', '')
    
    if 'application/pdf' in content_type:
        download_pdf(course_url, output_folder)
        return

    if 'text/html' not in content_type:
        return
    
    filename = create_filename(course_url, extension=".html")
    file_path = os.path.join(output_folder, filename)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"Saved webpage: {filename}")

def crawl_courses_page(page_url, output_folder, base_netloc):
    try:
        response = requests.get(page_url)
    except Exception as e:
        print(f"Failed to fetch {page_url}: {e}")
        return
    
    soup = BeautifulSoup(response.text, 'html.parser')

    for link in soup.find_all('a', href=True):
        href = link['href']
        if "course" in href:
            course_url = urljoin(page_url, href)
            if is_valid(course_url, base_netloc):
                crawl_course_page(course_url, output_folder)

def crawl_all_courses(base_url, output_folder, base_netloc, total_pages=20):
    for page_number in range(1, total_pages + 1):
        page_url = f"{base_url}?paged={page_number}#general-listing"
        print(f"Crawling page {page_number}: {page_url}")
        crawl_courses_page(page_url, output_folder, base_netloc)

output_folder = 'downloaded_docs'
os.makedirs(output_folder, exist_ok=True)
base_url = "https://www.sutd.edu.sg/education/undergraduate/courses"
base_netloc = urlparse(base_url).netloc

crawl_all_courses(base_url, output_folder, base_netloc, total_pages=20)

Crawling page 1: https://www.sutd.edu.sg/education/undergraduate/courses?paged=1#general-listing
Saved webpage: www.sutd.edu.sg_education_undergraduate_courses.html
Saved webpage: www.sutd.edu.sg_education_undergraduate_courses.html
Saved webpage: www.sutd.edu.sg_course_01-018-design-thinking-project-i_.html
Saved webpage: www.sutd.edu.sg_course_01-019-design-thinking-project-ii_.html
Saved webpage: www.sutd.edu.sg_course_01-020-design-thinking-project-iii_.html
Saved webpage: www.sutd.edu.sg_course_01-101-technologies-for-sustainable-global-health_.html
Saved webpage: www.sutd.edu.sg_course_01-102-energy-systems-and-management_.html
Saved webpage: www.sutd.edu.sg_course_01-106-engineering-management_.html
Saved webpage: www.sutd.edu.sg_course_01-107-urban-transportation_.html
Saved webpage: www.sutd.edu.sg_course_01-114-instructional-design-of-serious-games-for-healthcare_.html
Saved webpage: www.sutd.edu.sg_course_01-115-science-of-sound-acoustics-audio-music_.html
Saved webpage: www

# Split documents
Use LangChain to split the documents into smaller text chunks. 

In [3]:
# QUESTION: Use langchain to split the documents into chunks 

#--- ADD YOUR SOLUTION HERE (20 points)---
# Clean documents to remove whitespace + newline
def clean_text(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Process PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

download_folder = "downloaded_docs"
documents = []

# Process HTML
html_files = [
    os.path.join(download_folder, f)
    for f in os.listdir(download_folder)
    if f.lower().endswith(".html")
]
for file in html_files:
    if os.path.getsize(file) > 0:
        try:
            loader = UnstructuredHTMLLoader(file)
            docs = loader.load()
            for doc in docs:
                doc.page_content = clean_text(doc.page_content)
            documents.extend(docs)
        except Exception as e:
            print(f"Error processing {file}: {e}")
    else:
        print(f"Skipping empty file: {file}")

pdf_files = [
    os.path.join(download_folder, f)
    for f in os.listdir(download_folder)
    if f.lower().endswith(".pdf")
]
for file in pdf_files:
    text = extract_text_from_pdf(file)
    text = clean_text(text)
    document = Document(page_content=text)
    documents.append(document)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50
)
split_docs = text_splitter.split_documents(documents)
print(f"Total number of chunks created: {len(split_docs)}")

Skipping empty file: downloaded_docs/www.sutd.edu.sg_global_team.html
Total number of chunks created: 22071


### QUESTION: 

What chunking method or strategy did you use? Why did you use this method. Explain your design decision in less than 10 sentences.


**--- ADD YOUR SOLUTION HERE (10 points) ---**

We used LangChain's `RecursiveCharacterTextSplitter` to split the documents. For pre-processing, we designed appropriate data loaders for our two datatypes, PDF and HTML, as well as a function to clean the document before chunking. Then, it breaks text into manageable chunks (size 256) while including overlapping sections between chunks (size 50) to preserve context. The overlapping helps maintain continuity, ensuring that important details spanning chunk boundaries aren't lost. We chose this approach because it is flexible, allowing for adjustable chunk sizes and overlap lengths. This balance between chunk size and context retention is ideal for processing with language models that have token limits.

------------------------------


In [4]:
# QUESTION: create embeddings of document chunks and store them in a local vector store for fast lookup
# Decide an appropriate embedding model. Use Huggingface to run the embedding model locally.
# You do not have to use cloud-based APIs.

#--- ADD YOUR SOLUTION HERE (20 points)---
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = FAISS.from_documents(split_docs, embedding_model)
vector_store.save_local("faiss_index")
#------------------------------

/var/folders/8f/56qvs3bd6jsbf5sy0qgrzr680000gn/T/ipykernel_14573/4093568086.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


### QUESTION: 

What embeddings and vector store did you use and why? Explain your design decision in less than 10 sentences.


**--- ADD YOUR SOLUTION HERE (10 points) ---**<br>
We selected the `"sentence-transformers/all-mpnet-base-v2"` model for generating embeddings because it provides a strong balance between semantic accuracy and computational efficiency, capturing contextual nuances effectively. FAISS provides efficient similarity search capabilities for fast retrieval, and using LangChain’s wrappers simplifies integration. This combination ensures that our retrieval-augmented generation system can quickly and accurately fetch relevant document chunks.



In [ ]:
# Execute a query against the vector store

query = "When was SUTD founded?"

# QUESTION: run the query against the vector store, print the top 5 search results

#--- ADD YOUR SOLUTION HERE (5 points)---
# This is a hybrid search approach that leverages similarity search and BM25.
dense_results = vector_store.similarity_search(query, k=20)
candidate_texts = [doc.page_content for doc in dense_results]
tokenized_candidates = [word_tokenize(text.lower()) for text in candidate_texts]
tokenized_query = word_tokenize(query.lower())
bm25 = BM25Okapi(tokenized_candidates)
scores = bm25.get_scores(tokenized_query)
top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]

for rank, idx in enumerate(top_indices):
    print(f"Rank {rank+1} (BM25 score: {scores[idx]:.2f}):")
    print(candidate_texts[idx])
    print("-" * 50)
#------------------------------

Rank 1 (BM25 score: 4.51):
in school. SUTD has since witnessed over seven start-ups and even more in the pipeline. In 2017, SUTD was awarded the maximum accreditation tenure of five years across all of its undergraduate engineering and architecture programmes. This was granted
--------------------------------------------------
Rank 2 (BM25 score: 2.84):
of not going to University, back in 2017 when I caught the Start-up bug. But I am so happy to have joined SUTD. For the past 3.5 years, I have grown tremendously – with my feet planted on firm technical foundation, and my eyes on the stars. Lionell Jian
--------------------------------------------------
Rank 3 (BM25 score: 0.91):
Partnering with SUTD Giving Partnering with SUTD Giving to SUTD The philanthropic efforts of our fellow donors play a key role in our mission to nurture the next generations of leaders and innovators. Find out how our benefactors have paid it forward by
--------------------------------------------------
Rank 4 

In [ ]:
# QUESTION: Use the Huggingface transformers library to load the Llama 3.2-3B instruct model
# https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
# Run the model locally. You do not have to use cloud-based APIs.

# Execute the below query against the model and let it it answer from it's internal memory

query = "What courses are available in SUTD?"


#--- ADD YOUR SOLUTION HERE (40 points)---
login(token="")
model_name = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
print(f"Model is running on: {model.device}")
inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)
#------------------------------


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model is running on: cpu
What courses are available in SUTD? offered by Singapore-MIT Alliance for Research and Technology (SMART)
The Singapore-MIT Alliance for Research and Technology (SMART) offers a range of courses in various disciplines, including engineering, computer science, and life sciences. Here are some of the courses available in SMART:

**Engineering Courses:**

1. Master of Science in Engineering (MSE) in various disciplines, including:
	* Electrical Engineering and Computer Science (EECS)
	* Mechanical Engineering
	* Materials Science and Engineering
	* Biomedical Engineering
2. Master of Science in BioSystems and Micromechanics (MSc) in BioSystems and Micromechanics
3. Master of Science in Energy (MSE) in Energy
4. Master of Science in Engineering (MSE) in Environmental Engineering
5. Master of Science in Computer Science (MSCS) in Computer Science

**Computer Science Courses:**

1. Master of Science in Computer Science (MSCS) in Artificial Intelligence
2. Master of S

In [11]:
# QUESTION: Now put everything together. Use langchain to integrate your vector store and Llama model into a RAG system
# Run the below example question against your RAG system.

# Example questions
query = "How can I increase my chances of admission into SUTD?"


#--- ADD YOUR SOLUTION HERE (40 points)---
# Context retrieval
dense_results = vector_store.similarity_search(query, k=20)
candidate_texts = [doc.page_content for doc in dense_results]
tokenized_candidates = [word_tokenize(text.lower()) for text in candidate_texts]
tokenized_query = word_tokenize(query.lower())
bm25 = BM25Okapi(tokenized_candidates)
scores = bm25.get_scores(tokenized_query)
top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]

print("Context:")
context_chunks = []
for rank, idx in enumerate(top_indices):
    bm25_score = scores[idx]
    chunk = candidate_texts[idx]
    context_chunks.append(f"BM25 Score: {bm25_score:.2f}\n{chunk}")
    print(f"Chunk {rank+1}:")
    print(f"BM25 Score: {bm25_score:.2f}")
    print(chunk)
    print("-" * 50)
context = "\n\n".join(context_chunks)

print(f"Model is running on: {model.device}")

# RAG Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if device=="cuda" else -1,
    max_new_tokens=256,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.1,
)
llama_model = HuggingFacePipeline(pipeline=text_generation_pipeline)
prompt = (
    "Use the following pieces of context to answer the question at the end. "
    "If you don't know the answer, just say that you don't know, don't try to make up an answer. If the context is somewhat relevant, make your best effort to give a coherent answer. Ignore the BM25 score provided.\n\n"
    f"Context:\n{context}\n\n"
    f"Question: {query}\n"
    "Answer:"
)
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
final_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Final Answer:")
print(final_answer)
#------------------------------

Device set to use cpu


Context:
Chunk 1:
BM25 Score: 4.16
level Qualification level Qualification level Qualification level Application How to apply Application guide Applying to SUTD is as easy as 1-2-3. Just follow this step-by-step guide. Appeal If you would like to file an appeal, here's how. Admission
--------------------------------------------------
Chunk 2:
BM25 Score: 4.16
level Qualification level Qualification level Qualification level Application How to apply Application guide Applying to SUTD is as easy as 1-2-3. Just follow this step-by-step guide. Appeal If you would like to file an appeal, here's how. Admission
--------------------------------------------------
Chunk 3:
BM25 Score: 3.86
Be amongst leading experts and creative thinkers while you delve into design, AI, technology, and entrepreneurship. Go further with your potential and excel in the real world. Group of graduate students in the SUTD campus SUTD Academy Stay ahead in a
--------------------------------------------------
Chunk 4:


In [14]:
# QUESTION: Below is set of test questions. Add another 10 test questions based on your user interviews and your value proposition canvas.
# Run the complete set of test questions against the RAG question answering system. 

questions = ["What are the admissions deadlines for SUTD?",
             "Is there financial aid available?",
             "What is the minimum score for the Mother Tongue Language?",
             "Do I require reference letters?",
             "Can polytechnic diploma students apply?",
             "Do I need SAT score?",
             "How many PhD students does SUTD have?",
             "How much are the tuition fees for Singaporeans?",
             "How much are the tuition fees for international students?",
             "Is there a minimum CAP?",
            #  Added below:
            "What are the application requirements for international students?",
            "What is the difference between CSD and DAI?",
            "What kind of careers can I pursue with a degree in ESD from SUTD?",
            "What programs or majors does SUTD offer?",
            "What are the available student exchange programs at SUTD and which partner universities can I go to?",
            "Are interviews part of the admissions process?",
            "Are there scholarships for international students?",
            "What is campus life like at SUTD?",
            "Can an international student study with tuition grant at SUTD?",
            "Why is SUTD a Design AI university"
             ]

#--- ADD YOUR SOLUTION HERE (20 points)---
# Extra questions added to list above.

# Context retrieval
for query in questions:
    print(f"RUNNING FOR QUERY: {query}")
    dense_results = vector_store.similarity_search(query, k=20)
    candidate_texts = [doc.page_content for doc in dense_results]
    tokenized_candidates = [word_tokenize(text.lower()) for text in candidate_texts]
    tokenized_query = word_tokenize(query.lower())
    bm25 = BM25Okapi(tokenized_candidates)
    scores = bm25.get_scores(tokenized_query)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]

    print("Context:")
    context_chunks = []
    for rank, idx in enumerate(top_indices):
        bm25_score = scores[idx]
        chunk = candidate_texts[idx]
        context_chunks.append(f"BM25 Score: {bm25_score:.2f}\n{chunk}")
        print(f"Chunk {rank+1}:")
        print(f"BM25 Score: {bm25_score:.2f}")
        print(chunk)
        print("-" * 50)
    context = "\n\n".join(context_chunks)

    print(f"Model is running on: {model.device}")

# RAG Pipeline
    text_generation_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device=0 if device=="cuda" else -1,
        max_new_tokens=256,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.1,
    )
    llama_model = HuggingFacePipeline(pipeline=text_generation_pipeline)
    prompt = (
        "Use the following pieces of context to answer the question at the end. "
        "If you don't know the answer, just say that you don't know, don't try to make up an answer. If the context is somewhat relevant, make your best effort to give a coherent answer. Ignore the BM25 score provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}\n"
        "Answer:"
    )
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    final_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Final Answer:")
    print(final_answer)
    print("----------------------------------"*3)
    print("\n\n\n\n\n")
#---------------------------

RUNNING FOR QUERY: What are the admissions deadlines for SUTD?
Context:
Chunk 1:
BM25 Score: 3.73
Deadline There are two PhD intakes a year, in September and January of the following year. Application for the September 2025 intake is open from 11 November 2024 to 31 March 2025. Application Process Follow the steps in the SUTD Online Application Portal
--------------------------------------------------
Chunk 2:
BM25 Score: 3.20
following August. Each Academic Year is divided into three semesters where coursework is taken: September to December January to April May to August Research Areas of SUTD Faculty Members Potential applicants who are interested in the SUTD PhD Programme
--------------------------------------------------
Chunk 3:
BM25 Score: 2.04
life, SUTD welcomes students who would like to get plugged into the SUTD community even before the main cohort matriculates in September. The Early Matriculation Exercise is available to SC/PR students who have a place reserved in SUTD fo

Device set to use cpu


Final Answer:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. If the context is somewhat relevant, make your best effort to give a coherent answer. Ignore the BM25 score provided.

Context:
BM25 Score: 3.73
Deadline There are two PhD intakes a year, in September and January of the following year. Application for the September 2025 intake is open from 11 November 2024 to 31 March 2025. Application Process Follow the steps in the SUTD Online Application Portal

BM25 Score: 3.20
following August. Each Academic Year is divided into three semesters where coursework is taken: September to December January to April May to August Research Areas of SUTD Faculty Members Potential applicants who are interested in the SUTD PhD Programme

BM25 Score: 2.04
life, SUTD welcomes students who would like to get plugged into the SUTD community even before the main cohort matriculates in Septe

Device set to use cpu


Context:
Chunk 1:
BM25 Score: 5.50
aid for International Students? Financial aid for International Students are very limited. International Students may apply for Tuition Fee Loan and Study Loan to help defray your tuition fees. It is important to note that the loan amount is capped at the
--------------------------------------------------
Chunk 2:
BM25 Score: 4.21
have to repay the financial aid awards awarded to me? No repayment is required. However in the event that you withdraw or terminate your course of study in SUTD prematurely, you will be required to refund the monies. You will be required to inform the
--------------------------------------------------
Chunk 3:
BM25 Score: 2.91
cards only. Can I accept more than 1 financial aid award? Yes, students may be awarded more than one financial aid award depending on your financial situation. What if I had applied for financing schemes such as Post-Secondary Education Account (PSEA),
--------------------------------------------------

...

Context:
Chunk 1:
BM25 Score: 5.35
presentation skills, shaping you into a design innovator with critical thinking and a good understanding of societal needs. Helpful resources Learn more about what student life at SUTD could look like. Variable curriculum Customise your own path Here, we
--------------------------------------------------
Chunk 2:
BM25 Score: 4.85
Happenings About What's happening at SUTD Our dynamic and vibrant campus is always buzzing with fresh ideas, innovative projects and exciting events. Be part of this community. Discover what's new and get involved. Events What's coming up Engage with us
--------------------------------------------------
Chunk 3:
BM25 Score: 2.49
Campus Life Discover more to life at SUTD than academics! Explore our campus designed for interaction and collaboration, and join our supportive community of fellow students, staff, and faculty. Student experience A deep dive into our student life Find out
---------------------------------------------

Device set to use cpu


Context:
Chunk 1:
BM25 Score: 6.14
Tuition Grant for my undergraduate degree in SUTD? Yes, you will be eligible to take up the Tuition Grant for your undergraduate degree in SUTD. Details on Tuition Grant can be found here. What is the admission process? You will generally be notified by
--------------------------------------------------
Chunk 2:
BM25 Score: 5.91
aid for International Students? Financial aid for International Students are very limited. International Students may apply for Tuition Fee Loan and Study Loan to help defray your tuition fees. It is important to note that the loan amount is capped at the
--------------------------------------------------
Chunk 3:
BM25 Score: 5.17
to note that the loan amount is capped at the tuition fees for a Singapore Citizen. Outstanding International Students who have applied for SUTD scholarships may also be awarded a scholarship along with your admission offer. Students should fully consider
---------------------------------------------

Device set to use cpu


Context:
Chunk 1:
BM25 Score: 3.85
World’s First Design AI University 16 January 2025 SUTD Pivots Towards Artificial Intelligence With $50M Investment, Becoming World’s First Design AI University SUTD’s pivot to AI is premised on the principle that AI should no longer be viewed as a
--------------------------------------------------
Chunk 2:
BM25 Score: 3.85
World’s First Design AI University 16 January 2025 SUTD Pivots Towards Artificial Intelligence With $50M Investment, Becoming World’s First Design AI University SUTD’s pivot to AI is premised on the principle that AI should no longer be viewed as a
--------------------------------------------------
Chunk 3:
BM25 Score: 3.85
World’s First Design AI University 16 January 2025 SUTD Pivots Towards Artificial Intelligence With $50M Investment, Becoming World’s First Design AI University SUTD’s pivot to AI is premised on the principle that AI should no longer be viewed as a
--------------------------------------------------
Chunk 4:
BM25

### QUESTION: 


Manually inspect each answer, fact check whether the answer is correct (use Google or any other method) and check the retrieved documents

For each question, answer and context triple, record the following

- How accurate is the answer (1-5, 5 best)?
- How relevant is the retrieved context (1-5, 5 best)?
- How grounded is the answer in the retrieved context (instead of relying on the LLM's internal knowledge) (1-5, 5 best)?

**--- ADD YOUR SOLUTION HERE (20 points) ---**

Question 1:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 4

Question 2:
- Accuracy (1 - 5): 4
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 3

Question 3:
- Accuracy (1 - 5): 3
- Relevance (1 - 5): 1
- Grounded in context without relying on LLM (1 - 5): 4

Question 4:
- Accuracy (1 - 5): 5
- Relevance (1 - 5): 2
- Grounded in context without relying on LLM (1 - 5): 5

Question 5:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 1
- Grounded in context without relying on LLM (1 - 5): 5

Question 6:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 1
- Grounded in context without relying on LLM (1 - 5): 5

Question 7:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 2
- Grounded in context without relying on LLM (1 - 5): 4

Question 8:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 5

Question 9:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 5
- Grounded in context without relying on LLM (1 - 5): 5

Question 10:
- Accuracy (1 - 5): 2
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 4

Question 11:
- Accuracy (1 - 5): 5
- Relevance (1 - 5): 5
- Grounded in context without relying on LLM (1 - 5): 5

Question 12:
- Accuracy (1 - 5): 3
- Relevance (1 - 5): 5
- Grounded in context without relying on LLM (1 - 5): 4

Question 13:
- Accuracy (1 - 5): 4
- Relevance (1 - 5): 5
- Grounded in context without relying on LLM (1 - 5): 5

Question 14:
- Accuracy (1 - 5): 4
- Relevance (1 - 5): 3
- Grounded in context without relying on LLM (1 - 5): 2

Question 15:
- Accuracy (1 - 5): 1
- Relevance (1 - 5): 3
- Grounded in context without relying on LLM (1 - 5): 2

Question 16:
- Accuracy (1 - 5): 5
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 5

Question 17:
- Accuracy (1 - 5): 3
- Relevance (1 - 5): 5
- Grounded in context without relying on LLM (1 - 5): 5

Question 18:
- Accuracy (1 - 5): 5
- Relevance (1 - 5): 5
- Grounded in context without relying on LLM (1 - 5): 5

Question 19:
- Accuracy (1 - 5): 2
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 3

Question 20:
- Accuracy (1 - 5): 5
- Relevance (1 - 5): 4
- Grounded in context without relying on LLM (1 - 5): 4
------------------------------



You can try improve the chatbot by going back to previous steps in the notebook and change things until the submission deadline. For example, you can add more data sources, change the embedding models, change the data pre-processing, etc. 


# End

This concludes assignment 3.

Please submit this notebook with your answers and the generated output cells as a **Jupyter notebook file** via github.


Every group member should do the following submission steps:
1. Create a private github repository **sutd_5055mlop** under your github user.
2. Add your instructors as collaborator: ddahlmeier and lucainiaoge
3. Save your submission as assignment_03_GROUP_NAME.ipynb where GROUP_NAME is the name of the group you have registered. 
4. Push the submission files to your repo 
5. Submit the link to the repo via eDimensions



**Assignment due 6 April 2025 11:59pm**